In [4]:
from fastai.vision.all import *

In [5]:
path = untar_data(URLs.PETS)
path.ls()


 Download of https://s3.amazonaws.com/fast-ai-imageclas/oxford-iiit-pet.tgz has failed after 5 retries
 Fix the download manually:
$ mkdir -p /home/jjmachan/.fastai/archive
$ cd /home/jjmachan/.fastai/archive
$ wget -c https://s3.amazonaws.com/fast-ai-imageclas/oxford-iiit-pet.tgz
$ tar xf oxford-iiit-pet.tgz
 And re-run your code once the download is successful

File downloaded is broken. Remove /home/jjmachan/.fastai/archive/oxford-iiit-pet.tgz and try again.


EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
files = get_image_files(path/"images")
len(files)

In [ ]:
def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, 
                                      label_func, 
                                      item_tfms=Rsize(224))

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_turn(1)